# Test D: Polarimetry texture form

This notebook evaluates polarimetric observations of **3I/ATLAS** for repeatable glints above a standard phase curve.

## Inputs

Upload a CSV file named `polarimetry.csv` with the following columns:

- `phase_deg`: phase angle (Sun–object–observer) in degrees.
- `P_percent`: measured linear polarisation degree (%) for each exposure.
- `P_err`: uncertainty in `P_percent` (optional, will assume equal weights if absent).
- Optional: `rotation_phase` giving the rotational phase of the nucleus for repeatability checks.

## Outputs

The notebook fits a linear baseline to the polarisation vs phase curve between 10° and 50°, computes residuals, identifies points **>3σ** above the baseline in the 20–40° phase range and reports whether at least two such glints occur at similar rotation phases.  A plot of the data and baseline is also produced.


In [ ]:
import sys
!pip install --quiet numpy pandas matplotlib > /dev/null

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load polarimetry data
df = pd.read_csv('polarimetry.csv')

phase = df['phase_deg'].values
P = df['P_percent'].values
P_err = df['P_err'].values if 'P_err' in df.columns else np.ones_like(P)

# Select phase range for fitting (10°–50°)
mask_fit = (phase >= 10) & (phase <= 50)

# Weighted linear fit using numpy.polyfit (degree 1)
coeffs = np.polyfit(phase[mask_fit], P[mask_fit], 1, w=1.0/P_err[mask_fit])
baseline = np.polyval(coeffs, phase)

# Compute residuals and sigma
residuals = P - baseline
sigma = np.nanstd(residuals)

# Identify glints: residual > 3σ in 20°–40°
glint_mask = (phase >= 20) & (phase <= 40) & (residuals > 3 * sigma)
glint_indices = np.where(glint_mask)[0]

# Plot the data and baseline
plt.figure(figsize=(8,4))
plt.errorbar(phase, P, yerr=P_err, fmt='o', ms=5, label='Data')
plt.plot(phase, baseline, 'r-', label='Baseline (linear fit)')
plt.xlabel('Phase angle (deg)')
plt.ylabel('Polarisation (%)')
plt.title('Polarisation vs phase')
plt.legend()
plt.show()

print(f'Number of >3σ residuals in 20–40°: {len(glint_indices)}')

# If rotation phase is provided, check for recurring glints
if 'rotation_phase' in df.columns and len(glint_indices) > 0:
    rot_phases = df.loc[glint_indices, 'rotation_phase'].values
    # Bin rotation phases into 0.05 cycles bins
    binned = np.floor(rot_phases / 0.05)
    counts = np.bincount(binned.astype(int))
    recurring = np.any(counts >= 2)
    if recurring:
        print('Repeatable glints detected at similar rotation phases.')
    else:
        print('No repeatable glints detected in rotation phase bins.')
else:
    if len(glint_indices) >= 2:
        print('Multiple glints detected; recurrence analysis unavailable without rotation phase.')
    else:
        print('No significant polarimetric glints detected.')
